In [2]:
import pandas as pd




df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Documentation\Notebooks\9_5.csv')
df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,Manual_Lat,Manual_Long,Final_Lat,Final_Long,Geocoding_API_geo_status,Geocoding_API_formatted_address,Geocoding_API_lat,Geocoding_API_lng,Geocoding_API_place_id,Geocoding_API_types
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,NaN,NaN,40.910948,-111.404529,NaN,NaN,NaN,NaN,NaN,NaN
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,NaN,NaN,40.910948,-111.404529,NaN,NaN,NaN,NaN,NaN,NaN
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,NaN,NaN,40.910948,-111.404529,NaN,NaN,NaN,NaN,NaN,NaN
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,NaN,NaN,40.910948,-111.404529,NaN,NaN,NaN,NaN,NaN,NaN
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,NaN,NaN,40.910948,-111.404529,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,NaN,NaN,39.906627,-122.195031,NaN,NaN,NaN,NaN,NaN,NaN
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,NaN,NaN,41.646968,-122.528414,NaN,NaN,NaN,NaN,NaN,NaN
2330,"Corning , 96021 Flying J Travel Plaza # 0510154",2 530-824-8770 I-5 Exit 630 ( South Ave ),124 HRS XL 24 HRS,Corning,96021,Flying J Travel Plaza # 0510154,530-824-8770,2008,Corning,CA,...,NaN,NaN,41.646968,-122.528414,NaN,NaN,NaN,NaN,NaN,NaN
2331,A Grenada ( 96038 ) 3 J's Food Mart ( 76 ),530-436-2208 I-5 Exit 766 ( CA A12 E ),<U+2610><U+2610> XL <U+2610>,Grenada,96038,3 J's Food Mart ( 76 ),530-436-2208,2014,Grenada,CA,...,NaN,NaN,41.646968,-122.528414,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df=df[df['Address_Type'] == 'Proper']

In [6]:
# Check column names to ensure we use the correct ones
print("Available columns:")
print([col for col in df.columns if 'lat' in col.lower() or 'lng' in col.lower() or 'lon' in col.lower() or 'final' in col.lower() or 'geocoding' in col.lower()])
print("\nDataframe shape:", df.shape)

Available columns:
['Webscraped_Phone_Latitude', 'Webscraped_Phone_Longitude', 'Webscraped_Phone_LD_Latitude', 'Webscraped_Phone_LD_Longitude', 'Webscraped_PlacedMatched_Latitude', 'Webscraped_PlacedMatched_Longitude', 'Webscraped_PlacedMatched_LD_Latitude', 'Webscraped_PlacedMatched_LD_Longitude', 'Yelp_Latitude', 'Yelp_Longitude', 'YellowPages_JSONLD_LAT_1', 'YellowPages_JSONLD_LNG_1', 'Mid_Lat', 'Mid_Long', 'Manual_Lat', 'Manual_Long', 'Final_Lat', 'Final_Long', 'Geocoding_API_geo_status', 'Geocoding_API_formatted_address', 'Geocoding_API_lat', 'Geocoding_API_lng', 'Geocoding_API_place_id', 'Geocoding_API_types']

Dataframe shape: (2333, 161)


In [8]:
import numpy as np
import plotly.graph_objects as go
from math import radians, cos, sin, asin, sqrt

# Define the columns and their labels for comparing Final coordinates with Geocoding API
columns_info = [
    ('Final_Lat', 'Final_Long', 'Final Coordinates'),
    ('Geocoding_API_lat', 'Geocoding_API_lng', 'Geocoding API')
]

def parse_coords(lat_str, lon_str):
    lats = [float(x) for x in str(lat_str).split(';') if x.strip().lower() not in ['nan', '', 'none', 'removed']]
    lons = [float(x) for x in str(lon_str).split(';') if x.strip().lower() not in ['nan', '', 'none', 'removed']]
    return list(zip(lats, lons))

fig = go.Figure()
min_dist = 2  # miles

for idx, row in df.iterrows():
    all_coords = []
    hover_texts = []
    for lat_col, lon_col, label in columns_info:
        if lat_col in row and lon_col in row:
            coords = parse_coords(row[lat_col], row[lon_col])
            if coords:
                all_coords.extend(coords)
                hover_texts.extend([f'Row: {idx}<br>Source: {label}'] * len(coords))
    
    # Check if at least one segment is greater than min_dist
    show_trace = False
    if len(all_coords) > 1:
        for i in range(1, len(all_coords)):
            lat1, lon1 = all_coords[i-1]
            lat2, lon2 = all_coords[i]
            R = 3958.8  # Earth's radius in miles
            dlat = radians(lat2 - lat1)
            dlon = radians(lon2 - lon1)
            a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
            c = 2 * asin(sqrt(a))
            d = R * c
            if d >= min_dist:
                show_trace = True
                break
    
    # If at least one segment is long enough, show the whole trace
    if show_trace and len(all_coords) > 1:
        lats, lons = zip(*all_coords)
        fig.add_trace(go.Scattermapbox(
            lat=lats,
            lon=lons,
            mode='markers+lines',
            marker=dict(size=8),
            line=dict(width=2),
            name=f'Row {idx}',
            showlegend=True,
            text=hover_texts,
            hoverinfo='text+lat+lon'
        ))
    # If only one point, do not show it (per your request)

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4,
    mapbox_center={"lat": 39.5, "lon": -98.35},
    margin={"r":0,"t":0,"l":0,"b":0},
    showlegend=True,
    title="Accuracy Comparison: Final Coordinates vs Geocoding API"
)
fig.write_html(r'C:\Users\clint\Desktop\Geocoding_Documentation\Data\final_vs_geocoding_api_map.html')
fig.show()

C:\Users\clint\AppData\Local\Temp\ipykernel_28084\1043520720.py:48: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [4]:
# Calculate accuracy statistics between Final coordinates and Geocoding API
import pandas as pd
from math import radians, cos, sin, asin, sqrt

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on Earth (in miles)"""
    if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
        return None
    
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 3958.8  # Radius of Earth in miles
    return c * r

# Calculate distances between Final and Geocoding API coordinates
df['Distance_Final_vs_API'] = df.apply(
    lambda row: haversine_distance(
        row['Final_Lat'], row['Final_Long'], 
        row['Geocoding_API_lat'], row['Geocoding_API_lng']
    ), axis=1
)

# Filter out rows where distance couldn't be calculated
valid_distances = df['Distance_Final_vs_API'].dropna()

print("Accuracy Statistics: Final Coordinates vs Geocoding API")
print("=" * 60)
print(f"Total comparisons: {len(valid_distances)}")
print(f"Mean distance: {valid_distances.mean():.2f} miles")
print(f"Median distance: {valid_distances.median():.2f} miles")
print(f"Standard deviation: {valid_distances.std():.2f} miles")
print(f"Min distance: {valid_distances.min():.2f} miles")
print(f"Max distance: {valid_distances.max():.2f} miles")
print()
print("Distance Distribution:")
print(f"Within 0.1 miles: {(valid_distances <= 0.1).sum()} ({(valid_distances <= 0.1).mean()*100:.1f}%)")
print(f"Within 0.5 miles: {(valid_distances <= 0.5).sum()} ({(valid_distances <= 0.5).mean()*100:.1f}%)")
print(f"Within 1 mile: {(valid_distances <= 1).sum()} ({(valid_distances <= 1).mean()*100:.1f}%)")
print(f"Within 2 miles: {(valid_distances <= 2).sum()} ({(valid_distances <= 2).mean()*100:.1f}%)")
print(f"Greater than 2 miles: {(valid_distances > 2).sum()} ({(valid_distances > 2).mean()*100:.1f}%)")

Accuracy Statistics: Final Coordinates vs Geocoding API
Total comparisons: 242
Mean distance: 325.06 miles
Median distance: 0.31 miles
Standard deviation: 673.65 miles
Min distance: 0.01 miles
Max distance: 2482.89 miles

Distance Distribution:
Within 0.1 miles: 102 (42.1%)
Within 0.5 miles: 124 (51.2%)
Within 1 mile: 124 (51.2%)
Within 2 miles: 131 (54.1%)
Greater than 2 miles: 111 (45.9%)


C:\Users\clint\AppData\Local\Temp\ipykernel_8760\400148559.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Distance_Final_vs_API'] = df.apply(


In [9]:
import numpy as np
import plotly.graph_objects as go
from math import radians, cos, sin, asin, sqrt

# Define the columns and their labels (order matters for connection)
columns_info = [
    ('Webscraped_Phone_Latitude', 'Webscraped_Phone_Longitude', 'Truckstopsandservices (Phone)'),
    ('Webscraped_PlacedMatched_Latitude', 'Webscraped_PlacedMatched_Longitude', 'truckstopsandservices (PlaceMatched)'),
    ('Yelp_Latitude', 'Yelp_Longitude', 'Yelp (Phone)'),
    ('YellowPages_JSONLD_LAT_1', 'YellowPages_JSONLD_LNG_1', 'YellowPages (Phone)')
    # Add more sources here if needed
 ]

def parse_coords(lat_str, lon_str):
    lats = [float(x) for x in str(lat_str).split(';') if x.strip().lower() not in ['nan', '', 'none', 'removed']]
    lons = [float(x) for x in str(lon_str).split(';') if x.strip().lower() not in ['nan', '', 'none', 'removed']]
    return list(zip(lats, lons))

fig = go.Figure()
min_dist = 2  # miles

for idx, row in df.iterrows():
    all_coords = []
    hover_texts = []
    for lat_col, lon_col, label in columns_info:
        if lat_col in row and lon_col in row:
            coords = parse_coords(row[lat_col], row[lon_col])
            if coords:
                all_coords.extend(coords)
                hover_texts.extend([f'Row: {idx}<br>Source: {label}'] * len(coords))
    # Check if at least one segment is greater than min_dist
    show_trace = False
    if len(all_coords) > 1:
        for i in range(1, len(all_coords)):
            lat1, lon1 = all_coords[i-1]
            lat2, lon2 = all_coords[i]
            R = 3958.8
            dlat = radians(lat2 - lat1)
            dlon = radians(lon2 - lon1)
            a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
            c = 2 * asin(sqrt(a))
            d = R * c
            if d >= min_dist:
                show_trace = True
                break
    # If at least one segment is long enough, show the whole trace
    if show_trace and len(all_coords) > 1:
        lats, lons = zip(*all_coords)
        fig.add_trace(go.Scattermapbox(
            lat=lats,
            lon=lons,
            mode='markers+lines',
            marker=dict(size=8),
            line=dict(width=2),
            name=f'Row {idx}',
            showlegend=True,
            text=hover_texts,
            hoverinfo='text+lat+lon'
        ))
    # If only one point, do not show it (per your request)

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4,
    mapbox_center={"lat": 39.5, "lon": -98.35},
    margin={"r":0,"t":0,"l":0,"b":0},
    showlegend=True
)
fig.write_html(r'C:\Users\clint\Desktop\Geocoding_Documentation\Data\webscraped_map_3.html')

C:\Users\clint\AppData\Local\Temp\ipykernel_28084\648175396.py:49: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [11]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Get the valid distances (non-null values)
valid_distances = df['Distance_Final_vs_API'].dropna()

# Create subplots with multiple distribution visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Histogram - Full Range', 'Histogram - Zoomed (0-10 miles)', 
                   'Box Plot', 'Cumulative Distribution'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Histogram - Full range
fig.add_trace(
    go.Histogram(x=valid_distances, nbinsx=50, name='Full Range'),
    row=1, col=1
)

# 2. Histogram - Zoomed to 0-10 miles
distances_zoomed = valid_distances[valid_distances <= 10]
fig.add_trace(
    go.Histogram(x=distances_zoomed, nbinsx=50, name='0-10 miles'),
    row=1, col=2
)

# 3. Box plot
fig.add_trace(
    go.Box(y=valid_distances, name='Distance Distribution'),
    row=2, col=1
)

# 4. Cumulative distribution
sorted_distances = np.sort(valid_distances)
cumulative_prob = np.arange(1, len(sorted_distances) + 1) / len(sorted_distances)
fig.add_trace(
    go.Scatter(x=sorted_distances, y=cumulative_prob, mode='lines', name='Cumulative'),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Distribution of Distance_Final_vs_Geocoding_API (miles)",
    showlegend=False
)

# Update x-axis labels
fig.update_xaxes(title_text="Distance (miles)", row=1, col=1)
fig.update_xaxes(title_text="Distance (miles)", row=1, col=2)
fig.update_xaxes(title_text="", row=2, col=1)
fig.update_xaxes(title_text="Distance (miles)", row=2, col=2)

# Update y-axis labels
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Distance (miles)", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Probability", row=2, col=2)

fig.show()

# Additional summary statistics
print("Distribution Summary: Final Coordinates vs Geocoding API")
print("=" * 55)
print(f"Total observations: {len(valid_distances)}")
print(f"Mean: {valid_distances.mean():.2f} miles")
print(f"Median: {valid_distances.median():.2f} miles")
print(f"Mode (most frequent bin): {valid_distances.mode().iloc[0]:.2f} miles" if not valid_distances.mode().empty else "Mode: N/A")
print(f"Standard deviation: {valid_distances.std():.2f} miles")
print(f"Variance: {valid_distances.var():.2f}")
print(f"Skewness: {valid_distances.skew():.2f}")
print(f"Kurtosis: {valid_distances.kurtosis():.2f}")
print()
print("Percentiles:")
print(f"25th percentile: {valid_distances.quantile(0.25):.2f} miles")
print(f"50th percentile (median): {valid_distances.quantile(0.50):.2f} miles") 
print(f"75th percentile: {valid_distances.quantile(0.75):.2f} miles")
print(f"90th percentile: {valid_distances.quantile(0.90):.2f} miles")
print(f"95th percentile: {valid_distances.quantile(0.95):.2f} miles")
print(f"99th percentile: {valid_distances.quantile(0.99):.2f} miles")

Distribution Summary: Final Coordinates vs Geocoding API
Total observations: 242
Mean: 325.06 miles
Median: 0.31 miles
Mode (most frequent bin): 0.01 miles
Standard deviation: 673.65 miles
Variance: 453811.01
Skewness: 2.16
Kurtosis: 3.35

Percentiles:
25th percentile: 0.03 miles
50th percentile (median): 0.31 miles
75th percentile: 206.08 miles
90th percentile: 1733.57 miles
95th percentile: 2039.78 miles
99th percentile: 2461.00 miles


In [12]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Get the valid distances (non-null values)
valid_distances = df['Distance_Final_vs_API'].dropna()

# Remove outliers using IQR method
Q1 = valid_distances.quantile(0.25)
Q3 = valid_distances.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
distances_no_outliers = valid_distances[(valid_distances >= lower_bound) & (valid_distances <= upper_bound)]

print("Outlier Analysis:")
print("=" * 40)
print(f"Original data points: {len(valid_distances)}")
print(f"Q1 (25th percentile): {Q1:.2f} miles")
print(f"Q3 (75th percentile): {Q3:.2f} miles") 
print(f"IQR: {IQR:.2f} miles")
print(f"Lower bound (Q1 - 1.5*IQR): {lower_bound:.2f} miles")
print(f"Upper bound (Q3 + 1.5*IQR): {upper_bound:.2f} miles")
print(f"Data points after removing outliers: {len(distances_no_outliers)}")
print(f"Outliers removed: {len(valid_distances) - len(distances_no_outliers)} ({((len(valid_distances) - len(distances_no_outliers))/len(valid_distances)*100):.1f}%)")
print()

# Create subplots comparing with and without outliers
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Histogram - Without Outliers', 'Histogram - Zoomed (0-5 miles)', 
                   'Box Plot Comparison', 'Cumulative Distribution Comparison'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Histogram without outliers
fig.add_trace(
    go.Histogram(x=distances_no_outliers, nbinsx=30, name='No Outliers'),
    row=1, col=1
)

# 2. Histogram - Zoomed to 0-5 miles (no outliers)
distances_zoomed_no_outliers = distances_no_outliers[distances_no_outliers <= 5]
fig.add_trace(
    go.Histogram(x=distances_zoomed_no_outliers, nbinsx=30, name='0-5 miles (No Outliers)'),
    row=1, col=2
)

# 3. Box plot comparison
fig.add_trace(
    go.Box(y=valid_distances, name='With Outliers', boxpoints='outliers'),
    row=2, col=1
)
fig.add_trace(
    go.Box(y=distances_no_outliers, name='Without Outliers', boxpoints=False),
    row=2, col=1
)

# 4. Cumulative distribution comparison
sorted_distances_original = np.sort(valid_distances)
cumulative_prob_original = np.arange(1, len(sorted_distances_original) + 1) / len(sorted_distances_original)
fig.add_trace(
    go.Scatter(x=sorted_distances_original, y=cumulative_prob_original, mode='lines', 
              name='With Outliers', line=dict(color='blue')),
    row=2, col=2
)

sorted_distances_no_outliers = np.sort(distances_no_outliers)
cumulative_prob_no_outliers = np.arange(1, len(sorted_distances_no_outliers) + 1) / len(sorted_distances_no_outliers)
fig.add_trace(
    go.Scatter(x=sorted_distances_no_outliers, y=cumulative_prob_no_outliers, mode='lines', 
              name='Without Outliers', line=dict(color='red')),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Distribution Analysis: Final Coordinates vs Geocoding API (Outliers Removed)",
    showlegend=True
)

# Update x-axis labels
fig.update_xaxes(title_text="Distance (miles)", row=1, col=1)
fig.update_xaxes(title_text="Distance (miles)", row=1, col=2)
fig.update_xaxes(title_text="", row=2, col=1)
fig.update_xaxes(title_text="Distance (miles)", row=2, col=2)

# Update y-axis labels
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Distance (miles)", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Probability", row=2, col=2)

fig.show()

# Statistics comparison
print("Statistics Comparison:")
print("=" * 60)
print(f"{'Metric':<25} {'With Outliers':<15} {'Without Outliers':<15}")
print("-" * 60)
print(f"{'Count':<25} {len(valid_distances):<15} {len(distances_no_outliers):<15}")
print(f"{'Mean (miles)':<25} {valid_distances.mean():<15.2f} {distances_no_outliers.mean():<15.2f}")
print(f"{'Median (miles)':<25} {valid_distances.median():<15.2f} {distances_no_outliers.median():<15.2f}")
print(f"{'Std Dev (miles)':<25} {valid_distances.std():<15.2f} {distances_no_outliers.std():<15.2f}")
print(f"{'Min (miles)':<25} {valid_distances.min():<15.2f} {distances_no_outliers.min():<15.2f}")
print(f"{'Max (miles)':<25} {valid_distances.max():<15.2f} {distances_no_outliers.max():<15.2f}")
print(f"{'Skewness':<25} {valid_distances.skew():<15.2f} {distances_no_outliers.skew():<15.2f}")
print()

print("Accuracy Distribution (Without Outliers):")
print("-" * 45)
print(f"Within 0.1 miles: {(distances_no_outliers <= 0.1).sum()} ({(distances_no_outliers <= 0.1).mean()*100:.1f}%)")
print(f"Within 0.5 miles: {(distances_no_outliers <= 0.5).sum()} ({(distances_no_outliers <= 0.5).mean()*100:.1f}%)")
print(f"Within 1 mile: {(distances_no_outliers <= 1).sum()} ({(distances_no_outliers <= 1).mean()*100:.1f}%)")
print(f"Within 2 miles: {(distances_no_outliers <= 2).sum()} ({(distances_no_outliers <= 2).mean()*100:.1f}%)")
print(f"Within 5 miles: {(distances_no_outliers <= 5).sum()} ({(distances_no_outliers <= 5).mean()*100:.1f}%)")

Outlier Analysis:
Original data points: 242
Q1 (25th percentile): 0.03 miles
Q3 (75th percentile): 206.08 miles
IQR: 206.05 miles
Lower bound (Q1 - 1.5*IQR): -309.05 miles
Upper bound (Q3 + 1.5*IQR): 515.15 miles
Data points after removing outliers: 199
Outliers removed: 43 (17.8%)



Statistics Comparison:
Metric                    With Outliers   Without Outliers
------------------------------------------------------------
Count                     242             199            
Mean (miles)              325.06          40.50          
Median (miles)            0.31            0.08           
Std Dev (miles)           673.65          90.65          
Min (miles)               0.01            0.01           
Max (miles)               2482.89         488.81         
Skewness                  2.16            2.68           

Accuracy Distribution (Without Outliers):
---------------------------------------------
Within 0.1 miles: 102 (51.3%)
Within 0.5 miles: 124 (62.3%)
Within 1 mile: 124 (62.3%)
Within 2 miles: 131 (65.8%)
Within 5 miles: 138 (69.3%)


In [13]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Get the valid distances (non-null values)
valid_distances = df['Distance_Final_vs_API'].dropna()

# Filter to only distances within 1 mile (high-accuracy cases)
distances_within_1_mile = valid_distances[valid_distances <= 1]

print("High-Accuracy Analysis: Distances Within 1 Mile")
print("=" * 50)
print(f"Total data points: {len(valid_distances)}")
print(f"Points within 1 mile: {len(distances_within_1_mile)} ({(len(distances_within_1_mile)/len(valid_distances)*100):.1f}%)")
print(f"Points beyond 1 mile: {len(valid_distances) - len(distances_within_1_mile)} ({((len(valid_distances) - len(distances_within_1_mile))/len(valid_distances)*100):.1f}%)")
print()

# Create detailed analysis for the within-1-mile subset
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Histogram - Within 1 Mile', 'Histogram - Ultra-High Accuracy (0-0.1 miles)', 
                   'Box Plot - Within 1 Mile', 'Cumulative Distribution - Within 1 Mile'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Histogram - Within 1 mile
fig.add_trace(
    go.Histogram(x=distances_within_1_mile, nbinsx=25, name='Within 1 Mile'),
    row=1, col=1
)

# 2. Histogram - Ultra-high accuracy (0-0.1 miles)
distances_ultra_high = distances_within_1_mile[distances_within_1_mile <= 0.1]
fig.add_trace(
    go.Histogram(x=distances_ultra_high, nbinsx=20, name='0-0.1 miles'),
    row=1, col=2
)

# 3. Box plot - Within 1 mile
fig.add_trace(
    go.Box(y=distances_within_1_mile, name='Within 1 Mile', boxpoints='all'),
    row=2, col=1
)

# 4. Cumulative distribution - Within 1 mile
sorted_distances_1mile = np.sort(distances_within_1_mile)
cumulative_prob_1mile = np.arange(1, len(sorted_distances_1mile) + 1) / len(sorted_distances_1mile)
fig.add_trace(
    go.Scatter(x=sorted_distances_1mile, y=cumulative_prob_1mile, mode='lines', 
              name='Within 1 Mile', line=dict(color='green')),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="High-Accuracy Analysis: Final Coordinates vs Geocoding API (≤ 1 Mile)",
    showlegend=True
)

# Update x-axis labels
fig.update_xaxes(title_text="Distance (miles)", row=1, col=1)
fig.update_xaxes(title_text="Distance (miles)", row=1, col=2)
fig.update_xaxes(title_text="", row=2, col=1)
fig.update_xaxes(title_text="Distance (miles)", row=2, col=2)

# Update y-axis labels
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Distance (miles)", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Probability", row=2, col=2)

fig.show()

# Detailed statistics for within 1 mile
print("Detailed Statistics for High-Accuracy Cases (≤ 1 Mile):")
print("=" * 60)
print(f"Count: {len(distances_within_1_mile)}")
print(f"Mean: {distances_within_1_mile.mean():.4f} miles")
print(f"Median: {distances_within_1_mile.median():.4f} miles")
print(f"Standard deviation: {distances_within_1_mile.std():.4f} miles")
print(f"Min: {distances_within_1_mile.min():.4f} miles")
print(f"Max: {distances_within_1_mile.max():.4f} miles")
print(f"Skewness: {distances_within_1_mile.skew():.2f}")
print(f"Kurtosis: {distances_within_1_mile.kurtosis():.2f}")
print()

print("Precision Distribution (Within 1 Mile):")
print("-" * 45)
print(f"Within 0.01 miles (53 feet): {(distances_within_1_mile <= 0.01).sum()} ({(distances_within_1_mile <= 0.01).mean()*100:.1f}%)")
print(f"Within 0.05 miles (264 feet): {(distances_within_1_mile <= 0.05).sum()} ({(distances_within_1_mile <= 0.05).mean()*100:.1f}%)")
print(f"Within 0.1 miles (528 feet): {(distances_within_1_mile <= 0.1).sum()} ({(distances_within_1_mile <= 0.1).mean()*100:.1f}%)")
print(f"Within 0.25 miles (1,320 feet): {(distances_within_1_mile <= 0.25).sum()} ({(distances_within_1_mile <= 0.25).mean()*100:.1f}%)")
print(f"Within 0.5 miles (2,640 feet): {(distances_within_1_mile <= 0.5).sum()} ({(distances_within_1_mile <= 0.5).mean()*100:.1f}%)")
print(f"0.5 - 1.0 miles: {((distances_within_1_mile > 0.5) & (distances_within_1_mile <= 1.0)).sum()} ({((distances_within_1_mile > 0.5) & (distances_within_1_mile <= 1.0)).mean()*100:.1f}%)")
print()

print("Percentiles (Within 1 Mile):")
print("-" * 30)
percentiles = [10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    value = distances_within_1_mile.quantile(p/100)
    print(f"{p}th percentile: {value:.4f} miles ({value*5280:.0f} feet)")
print()

# Compare accuracy levels
print("Accuracy Level Breakdown:")
print("-" * 35)
total_within_1 = len(distances_within_1_mile)
print(f"Street-level accuracy (≤0.01 mi): {(distances_within_1_mile <= 0.01).sum()}/{total_within_1} ({(distances_within_1_mile <= 0.01).mean()*100:.1f}%)")
print(f"Block-level accuracy (0.01-0.1 mi): {((distances_within_1_mile > 0.01) & (distances_within_1_mile <= 0.1)).sum()}/{total_within_1} ({((distances_within_1_mile > 0.01) & (distances_within_1_mile <= 0.1)).mean()*100:.1f}%)")
print(f"Neighborhood accuracy (0.1-0.5 mi): {((distances_within_1_mile > 0.1) & (distances_within_1_mile <= 0.5)).sum()}/{total_within_1} ({((distances_within_1_mile > 0.1) & (distances_within_1_mile <= 0.5)).mean()*100:.1f}%)")
print(f"Local area accuracy (0.5-1.0 mi): {((distances_within_1_mile > 0.5) & (distances_within_1_mile <= 1.0)).sum()}/{total_within_1} ({((distances_within_1_mile > 0.5) & (distances_within_1_mile <= 1.0)).mean()*100:.1f}%)")

High-Accuracy Analysis: Distances Within 1 Mile
Total data points: 242
Points within 1 mile: 124 (51.2%)
Points beyond 1 mile: 118 (48.8%)



Detailed Statistics for High-Accuracy Cases (≤ 1 Mile):
Count: 124
Mean: 0.0591 miles
Median: 0.0307 miles
Standard deviation: 0.0766 miles
Min: 0.0060 miles
Max: 0.3462 miles
Skewness: 2.22
Kurtosis: 4.50

Precision Distribution (Within 1 Mile):
---------------------------------------------
Within 0.01 miles (53 feet): 17 (13.7%)
Within 0.05 miles (264 feet): 88 (71.0%)
Within 0.1 miles (528 feet): 102 (82.3%)
Within 0.25 miles (1,320 feet): 117 (94.4%)
Within 0.5 miles (2,640 feet): 124 (100.0%)
0.5 - 1.0 miles: 0 (0.0%)

Percentiles (Within 1 Mile):
------------------------------
10th percentile: 0.0089 miles (47 feet)
25th percentile: 0.0162 miles (86 feet)
50th percentile: 0.0307 miles (162 feet)
75th percentile: 0.0560 miles (296 feet)
90th percentile: 0.1755 miles (927 feet)
95th percentile: 0.2576 miles (1360 feet)
99th percentile: 0.3462 miles (1828 feet)

Accuracy Level Breakdown:
-----------------------------------
Street-level accuracy (≤0.01 mi): 17/124 (13.7%)
Block-level